In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
from tqdm.notebook import tqdm, trange

"""Change to the data folder"""
new_path = "./new_train/new_train"

cuda_status = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 64

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, out, scene_ids, track_ids, agent_ids]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, scene_ids, track_ids, agent_ids]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [4]:
model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(4, 8, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    #torch.nn.Conv2d(8, 24, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    #torch.nn.Linear(24 * 5 * 5, 512),
    #torch.nn.ReLU(),
    #torch.nn.Linear(512, 256),
    #torch.nn.ReLU(),
    #torch.nn.Linear(256, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
class LSTM(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = torch.nn.LSTM(76, hidden_size=98, batch_first=True)
        self.linear = torch.nn.Linear(19*98, 7200)
        self.linear2 = torch.nn.Linear(5880, 6500)
        self.relu = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(6500, 6500)
        self.linear4 = torch.nn.Linear(6500, 7200)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = torch.flatten(out, start_dim=1)
        #predict = self.linear(out)
        #predict = torch.reshape(predict, torch.Size([64, 60, 30, 4]))
        out = self.relu(self.linear2(out))
        out = self.relu(self.linear3(out))
        out = self.relu(self.linear4(out))
        return out

model = LSTM()
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
model = torch.load('./models/1epochAdam.pt')
model.train()
model.to(device)
if cuda_status:
    model = model.cuda()

### Visualize the batch of sequences

In [6]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 3

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out, scene_ids, track_ids, agent_ids = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])
        
# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        x = torch.flatten(inp, start_dim=2)

        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None

        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        loss = torch.sqrt(loss)
        print(loss.item())

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


595.5176391601562
518.7504272460938
350.3236999511719
471.8230285644531
358.3848571777344
260.1774597167969
195.37254333496094
214.919921875
183.5379180908203
169.55279541015625
173.09649658203125
214.51007080078125
68.60237884521484
128.9435272216797
162.61798095703125
75.94810485839844
175.1658935546875
30.801687240600586
130.76414489746094
104.53498840332031
69.88712310791016
129.19036865234375
38.41622543334961
128.32618713378906
27.962196350097656
88.03697967529297
45.86211013793945
109.01971435546875
23.50116539001465
57.83266067504883
78.68704986572266
78.75364685058594
63.00243377685547
60.8730354309082
41.0143928527832
85.49250030517578
32.722904205322266
93.15188598632812
36.73838424682617
72.16399383544922
58.88746643066406
90.59579467773438
23.912216186523438
72.8470458984375
37.87156295776367
80.13037109375
40.00065231323242
74.3509521484375
27.84984588623047
80.3686294555664
48.223270416259766
86.92333221435547
32.940757751464844
73.53670501708984
38.266963958740234
89.91

46.28207778930664
8.439879417419434
36.185123443603516
28.182039260864258
40.782020568847656
31.441625595092773
38.692710876464844
16.637990951538086
40.75779724121094
23.53154754638672
46.14322280883789
19.3338680267334
48.50819396972656
11.660279273986816
48.298709869384766
16.1064510345459
46.28834533691406
12.062610626220703
40.031768798828125
14.805492401123047
44.59483337402344
19.462018966674805
39.121826171875
7.300034999847412
24.01363182067871
43.76752853393555
34.525413513183594
27.743408203125
37.380027770996094
31.814687728881836
35.83980941772461
30.243083953857422
34.5100212097168
15.853996276855469
44.888893127441406
24.198623657226562
45.424259185791016
9.43684196472168
31.03265953063965
23.85213851928711
38.30720901489258
21.956518173217773
36.78919982910156
22.110610961914062
45.17974853515625
15.175603866577148
47.85655975341797
10.845549583435059
45.29816436767578
8.79875659942627
34.86021041870117
20.093631744384766
47.26080322265625
7.704500198364258
32.954574584

26.8977108001709
33.80451202392578
29.74704360961914
30.3781681060791
16.861112594604492
37.59554672241211
19.222291946411133
28.791818618774414
16.222827911376953
34.60624313354492
19.64168930053711
36.27064514160156
15.063058853149414
41.396732330322266
19.78878402709961
35.48930358886719
26.757442474365234
28.175373077392578
24.313020706176758
37.03563690185547
12.627825736999512
45.21857833862305
13.851129531860352
41.408958435058594
15.876930236816406
42.01236343383789
19.790382385253906
50.65245056152344
11.925741195678711
37.68379592895508
4.43537712097168
25.190019607543945
30.975366592407227
32.829917907714844
17.344572067260742
40.21299362182617
9.542991638183594
33.00434494018555
12.607256889343262
41.80349349975586
15.880282402038574
39.30500411987305
15.51285457611084
40.75192642211914
13.229883193969727
31.882287979125977
27.438199996948242
33.973567962646484
29.070894241333008
36.794151306152344
28.834184646606445
24.119869232177734
25.410261154174805
29.058839797973633


35.213253021240234
19.307268142700195
29.230796813964844
17.972858428955078
26.012039184570312
22.3624210357666
25.476634979248047
23.37856674194336
33.396278381347656
24.8433895111084
34.63470458984375
18.919574737548828
28.691280364990234
21.541948318481445
41.863739013671875
16.94753074645996
33.944339752197266
14.8948335647583
36.511539459228516
16.079200744628906
32.023929595947266
14.616643905639648
32.337345123291016
23.120147705078125
29.61158561706543
15.312162399291992
41.98564910888672
5.50853967666626
26.26788902282715
24.183944702148438
26.371023178100586
31.135372161865234
22.832170486450195
40.87346649169922
10.711368560791016
39.78963088989258
8.727266311645508
35.992000579833984
18.287145614624023
28.44399642944336
20.96584129333496
26.517826080322266
21.406251907348633
39.74195098876953
13.391550064086914
35.209754943847656
21.812211990356445
30.6662540435791
22.486417770385742
27.297819137573242
23.715227127075195
29.011791229248047
24.665061950683594
34.355705261230

25.60818862915039
13.394283294677734
26.867509841918945
22.90096092224121
24.738338470458984
25.997947692871094
27.721450805664062
20.896265029907227
34.585845947265625
17.643468856811523
25.717138290405273
21.80167579650879
26.812183380126953
20.102153778076172
32.64482116699219
20.532028198242188
26.101999282836914
20.446393966674805
30.59779930114746
9.494751930236816
26.054203033447266
16.09580421447754
30.03498649597168
12.540115356445312
31.465984344482422
7.405955791473389
28.975770950317383
19.2502498626709
30.31793975830078
11.192266464233398
31.905546188354492
19.87421226501465
29.142004013061523
17.692472457885742
30.40207862854004
20.22066307067871
25.438343048095703
27.84687614440918
29.786006927490234
19.731918334960938
33.00468444824219
27.061870574951172
28.765180587768555
12.371057510375977
30.954694747924805
18.133249282836914
32.58881759643555
12.838953018188477
33.211544036865234
22.80354881286621
20.786584854125977
31.006412506103516
22.95780372619629
32.7454872131

...

21.4864559173584
9.873369216918945
15.985414505004883
10.832572937011719
14.417308807373047
11.747172355651855
18.069110870361328
11.629948616027832
16.51857566833496
14.48609447479248
12.642477989196777
14.366971969604492
13.562095642089844
14.16628360748291
16.97157096862793
16.996517181396484
17.310060501098633
12.024194717407227
11.967395782470703
15.798147201538086
18.31348991394043
14.648662567138672
14.807442665100098
13.493988037109375
18.29271697998047
11.409502983093262
17.180618286132812
8.673980712890625
19.13686752319336
12.730985641479492
17.06989288330078
8.9564208984375
21.863718032836914
3.1209421157836914
11.780136108398438
11.292614936828613
16.19009780883789
13.286222457885742
18.268308639526367
17.11054039001465
15.146900177001953
16.278614044189453
11.052759170532227
12.606277465820312
15.232075691223145
9.397686958312988
19.278823852539062
5.861858367919922
19.61968421936035
12.920731544494629
16.343643188476562
9.57571792602539
20.631977081298828
5.9810585975646

17.899658203125
9.40440845489502
16.132043838500977
14.217649459838867
15.436402320861816
11.600654602050781
18.54608154296875
7.8750152587890625
14.968427658081055
7.997065544128418
21.18132972717285
5.444719314575195
22.811189651489258
6.514308929443359
19.797277450561523
10.707226753234863
18.776212692260742
6.825650691986084
17.37550926208496
10.036449432373047
19.831710815429688
9.678683280944824
21.918607711791992
4.145171165466309
21.226341247558594
8.904726028442383
13.486846923828125
8.17460823059082
17.716440200805664
10.229897499084473
17.806766510009766
9.756468772888184
12.15327262878418
15.508913040161133
11.567541122436523
15.937952041625977
7.6941351890563965
18.569217681884766
12.885784149169922
14.5629301071167
12.16781997680664
15.252346992492676
13.168610572814941
13.389787673950195
16.773061752319336
16.478078842163086
11.220833778381348
18.01103973388672
15.709996223449707
17.206867218017578
6.613095283508301
18.804401397705078
9.485597610473633
20.489315032958984

13.505184173583984
15.145330429077148
9.098734855651855
17.683300018310547
12.681142807006836
11.942285537719727
14.705928802490234
10.849325180053711
19.025470733642578
10.722939491271973
17.456363677978516
11.984549522399902
14.898319244384766
14.367640495300293
17.854183197021484
12.197649002075195
16.48641586303711
14.228132247924805
12.516047477722168
12.674972534179688
19.170169830322266
8.742992401123047
14.82038688659668
14.738608360290527
12.942717552185059
9.143960952758789
17.540578842163086
10.609476089477539
15.765775680541992
8.547093391418457
17.757047653198242
6.588712215423584
18.252317428588867
9.342040061950684
15.828694343566895
10.44375228881836
16.50904655456543
9.962846755981445
18.770376205444336
12.869542121887207
15.292524337768555
13.237778663635254
15.306806564331055
10.922475814819336
16.263456344604492
10.751160621643066
14.918782234191895
9.754952430725098
18.71693229675293
9.917462348937988
17.353321075439453
8.720942497253418
14.719512939453125
12.96359

7.120171070098877
17.922954559326172
7.772352695465088
17.325576782226562
7.728084087371826
14.894978523254395
9.614907264709473
16.50470542907715
12.476367950439453
16.43975257873535
8.475576400756836
17.168397903442383
11.96102523803711
15.984654426574707
11.811199188232422
22.26247215270996
9.206243515014648
18.564367294311523
11.770718574523926
16.49119758605957
11.216054916381836
13.363691329956055
10.240222930908203
16.332504272460938
8.229671478271484
17.964218139648438
7.397693157196045
22.33977508544922
11.269124984741211
16.43527603149414
13.969915390014648
14.321573257446289
15.244935035705566
10.440062522888184
17.593822479248047
13.027743339538574
15.360696792602539
16.183591842651367
11.098997116088867
16.296588897705078
7.839303970336914
14.147294998168945
10.85300350189209
15.999774932861328
11.979421615600586
14.234807968139648
11.552680969238281
13.89344596862793
9.375153541564941
13.580381393432617
13.402746200561523
16.817657470703125
16.587398529052734
16.916006088

13.193522453308105
17.68488311767578
7.516947269439697
19.28411865234375
6.631808757781982
19.387094497680664
9.759186744689941
15.026904106140137
13.884410858154297
13.325935363769531
15.02768325805664
13.810629844665527
14.244483947753906
15.238351821899414
14.337815284729004
15.552027702331543
9.689080238342285
15.132210731506348
7.186839580535889
16.29486656188965
7.551083564758301
18.758317947387695
5.915826797485352
16.45699691772461
7.08294677734375
17.938940048217773
9.002881050109863
16.208173751831055
7.30322790145874
16.356338500976562
6.649960517883301
18.398569107055664
8.877555847167969
18.045658111572266
8.376441955566406
17.434986114501953
10.640861511230469
11.490188598632812
15.125722885131836
17.199289321899414
5.1392388343811035
16.266389846801758
10.205132484436035
15.045831680297852
9.2868070602417
15.580253601074219
10.670269966125488
16.52410888671875
9.183772087097168
18.92286491394043
9.27044677734375
16.206323623657227
12.856078147888184
12.86277961730957
11.

In [ ]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 10
        
# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
#learning_rate =0.01
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        #x = torch.flatten(inp, start_dim=2)
        x = inp.permute(0,2,1,3).float()
        x = torch.flatten(x, start_dim=2)
        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None
        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        #y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        print(torch.sqrt(loss))

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


In [ ]:
torch.save(model, './models/12InitialWorkspace.pt')

In [2]:
model = torch.load('./models/12InitialWorkspace.pt')
model.eval()
model.to(device)
if cuda_status:
    model = model.cuda()

In [7]:
import pandas as pd

# Submission output
writeCSV = True
val_path = "./new_val_in/new_val_in"

if writeCSV:
    
    dataset = ArgoverseDataset(data_path=val_path)
    test_loader = DataLoader(dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)
    
    data = []
    
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(tqdm(test_loader)):
            inp, scene_ids, track_ids, agent_ids = sample_batch
            inp = torch.flatten(inp, start_dim=2)

            if cuda_status:
                model = model.cuda()
                x = inp.cuda()
            else:
                x = inp

            y_pred = None

            # Forward pass: predict y by passing x to the model.    
            y_pred = model(x.float())
            y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))
            
            for i in range(batch_sz):
                row = []
                row.append(scene_ids[i].item())
                curr = y_pred[i]
                
                agent_id = agent_ids[i]
                
                for j in range(30):
                    vehicle_index = 0
                    found = False
                    while not found:
                        if track_ids[i][vehicle_index][j][0] == agent_id:
                            found = True
                        else:
                            vehicle_index += 1

                    row.append(str(curr[vehicle_index][j][0].item()))
                    row.append(str(curr[vehicle_index][j][1].item()))
                    
                data.append(row)

    df = pd.DataFrame(data, columns = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35','v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52','v53','v54','v55','v56','v57','v58','v59','v60'])
    print(df)
    df.to_csv('submission.csv', index=False)
                
                
                


         ID                  v1                  v2                  v3  \
0     10002   1830.747802734375      535.7841796875   1744.435302734375   
1     10015   729.9448852539062   1250.381591796875     738.62451171875   
2     10019   566.5105590820312  1277.9290771484375   573.7442626953125   
3     10028  1810.7493896484375   522.7767333984375  1721.6607666015625   
4      1003   2149.365478515625   730.1478271484375   2123.237548828125   
...     ...                 ...                 ...                 ...   
3195   9897   260.5370788574219    827.177001953125  258.47503662109375   
3196     99   581.5123901367188   1175.595458984375   588.9920654296875   
3197   9905  1820.1151123046875   566.4986572265625  1767.1998291015625   
3198   9910    565.965576171875  1324.4290771484375   571.4410400390625   
3199   9918   580.6732177734375   1183.749755859375   586.4964599609375   

                      v4                 v5                 v6  \
0     441.82391357421875   1782.